In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import pandas as pd
import os
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import pandas as pd
import os
from PIL import Image

class DenoisingAutoencoder(nn.Module):
    def __init__(self):
        super(DenoisingAutoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 3, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid(),  
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [29]:
# Define transformations
transform = transforms.Compose([
    transforms.GaussianBlur(5),
    transforms.Resize((64, 64)),  # Resize images
    # transforms.ToTensor(),        # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize
])

to_tensor = transforms.ToTensor()

dae_path = "/kaggle/input/fungi-dae/pytorch/1/1/dae.pth"

# Custom dataset class for loading training data from CSV
class FungiDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)  # Read CSV file
        self.root_dir = root_dir  # Use the correct root directory
        self.transform = transform
        self.data = self.data[~self.data["ClassId"].isin([1, 3])]
        #self.dae = DenoisingAutoencoder()
        #self.dae.load_state_dict(torch.load(dae_path, weights_only=True))
        #self.dae.eval()

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 0])  # Combine root and image path
        
        # Debugging: Check if the file exists
        if not os.path.exists(img_path):
            print(f"⚠️ Warning: File not found - {img_path}")  
        
        image = Image.open(img_path).convert('RGB')  # Load image
        label = int(self.data.iloc[idx, 1])  # Get label
        image = to_tensor(image)

        """
        if label in [1, 3]:
            with torch.no_grad():
                image = self.dae(image)
        """

        if self.transform:
            image = self.transform(image)
        

        return image, label

    def __len__(self):
        return len(self.data)

# Load the full dataset from your CSV file
full_dataset = FungiDataset(
    csv_file="/kaggle/input/ds-3-datathon-2025-fungi-classification/DatathonFiles/fungi_train.csv",
    root_dir="/kaggle/input/ds-3-datathon-2025-fungi-classification/DatathonFiles",
    transform=transform
)
full_loader = DataLoader(full_dataset, batch_size=32, shuffle=True) # comment this out if you want validation set

# Split the dataset into training and validation sets (80% train, 20% val)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)


In [30]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        # For input 64x64, after three poolings: 64 -> 32 -> 16 -> 8, so 128 * 8 * 8 = 8192
        self.fc1 = nn.Linear(128 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = torch.flatten(x, 1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Use GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = SimpleCNN(num_classes=5).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


Using device: cuda


In [31]:
# Training loop with validation
num_epochs = 15
for epoch in range(num_epochs):
    # ---- Training ----
    model.train()
    running_train_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(full_loader, desc=f'Training Epoch {epoch+1}/{num_epochs}'):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_train_loss += loss.item() * images.size(0)
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)
    
    avg_train_loss = running_train_loss / total_train
    train_accuracy = 100 * correct_train / total_train
    """
    # ---- Validation ----; commented out to train on full train set
    
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc="Validation", leave=False):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            
            running_val_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
    
    avg_val_loss = running_val_loss / total_val
    val_accuracy = 100 * correct_val / total_val

    print(f"Epoch [{epoch+1}/{num_epochs}]: "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
    """
    print(f"Epoch [{epoch+1}/{num_epochs}]: "
          f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}%")
# Save the trained model
torch.save(model.state_dict(), "fungi.pth")
print("Model training completed and saved as 'fungi.pth'.")

Training Epoch 1/15: 100%|██████████| 94/94 [00:23<00:00,  4.01it/s]


Epoch [1/15]: Train Loss: 0.6702, Train Acc: 72.97%


Training Epoch 2/15: 100%|██████████| 94/94 [00:20<00:00,  4.52it/s]


Epoch [2/15]: Train Loss: 0.4455, Train Acc: 83.73%


Training Epoch 3/15: 100%|██████████| 94/94 [00:20<00:00,  4.64it/s]


...

Epoch [13/15]: Train Loss: 0.1309, Train Acc: 94.17%


Training Epoch 14/15: 100%|██████████| 94/94 [00:19<00:00,  4.73it/s]


Epoch [14/15]: Train Loss: 0.1079, Train Acc: 95.13%


Training Epoch 15/15: 100%|██████████| 94/94 [00:20<00:00,  4.67it/s]

Epoch [15/15]: Train Loss: 0.1483, Train Acc: 93.80%
Model training completed and saved as 'fungi.pth'.


In [32]:
# Custom dataset class for loading training data from CSV
class TestDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.data = pd.read_csv(csv_file)  # Read CSV file
        self.root_dir = root_dir  # Use the correct root directory
        self.transform = transform

    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir, self.data.iloc[idx, 1])  # Combine root and image path
        
        # Debugging: Check if the file exists
        if not os.path.exists(img_path):
            print(f"⚠️ Warning: File not found - {img_path}")  
        
        image = Image.open(img_path).convert('RGB')  # Load image
        id = int(self.data.iloc[idx, 0])  # Get id
        
        if self.transform:
            image = to_tensor(image)
            image = self.transform(image)

        return image, id

    def __len__(self):
        return len(self.data)


In [33]:
test_set = TestDataset(
    csv_file="/kaggle/input/ds-3-datathon-2025-fungi-classification/DatathonFiles/fungi_test.csv",
    root_dir="/kaggle/input/ds-3-datathon-2025-fungi-classification/DatathonFiles",
    transform=transform
)
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)

results = []
model.eval()
with torch.no_grad():  # Disable gradient computation for inference
    for images, ids in test_loader:
        outputs = model(images.to(device))
        _, predicted = torch.max(outputs, 1)  # Get the class with highest probability

        for img_id, pred in zip(ids, predicted):
            results.append({"id": img_id.item(), "output": pred.item()})

# Save to CSV
df = pd.DataFrame(results)
df.to_csv("fungi_submission.csv", index=False)

print("Inference completed. Results saved to fungi_submission.csv.")

Inference completed. Results saved to fungi_submission.csv.
